# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
weather_df = pd.read_csv("../output_data/weather_data.csv")
weather_df.head()


,city,country,date,lat,lng,temp,humd,wind,cloud
0,Port Alfred,ZA,1613178619,-33.5906,26.8910,75.00,95,5.01,73
1,Garden City,US,1613178750,42.3256,-83.3310,24.80,79,4.61,90
2,Jamestown,US,1613178482,42.0970,-79.2353,15.80,73,5.75,75
3,Atuona,PF,1613178752,-9.8000,-139.0333,79.14,78,14.56,4
4,Bubaque,GW,1613178753,11.2833,-15.8333,78.40,63,11.25,0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
humidity = weather_df["humd"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["lat", "lng"]]

In [20]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity,point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
fit_weather_df = weather_df.loc[(weather_df["temp"] > 70) & (weather_df["temp"] < 80) & (weather_df["cloud"] < 50), :]
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.reset_index(inplace=True)
del fit_weather_df['index']
fit_weather_df

,city,country,date,lat,lng,temp,humd,wind,cloud
0,Atuona,PF,1613178752,-9.8000,-139.0333,79.14,78,14.56,4
1,Bubaque,GW,1613178753,11.2833,-15.8333,78.40,63,11.25,0
2,Souillac,MU,1613178768,-20.5167,57.5167,78.80,83,10.36,40
3,Port Elizabeth,ZA,1613178780,-33.9180,25.5701,75.20,94,14.97,40
4,Kapaa,US,1613178796,22.0752,-159.3190,78.01,57,8.05,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotels = []

for i in range(len(fit_weather_df)):
    lat = fit_weather_df.loc[i]['lat']
    lng = fit_weather_df.loc[i]['lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotels",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
fit_weather_df["hotel name"] = hotels
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.head()


,city,country,date,lat,lng,temp,humd,wind,cloud,hotel name
0,Atuona,PF,1613178752,-9.8000,-139.0333,79.14,78,14.56,4,Atuona
1,Bubaque,GW,1613178753,11.2833,-15.8333,78.40,63,11.25,0,Bubaque
2,Souillac,MU,1613178768,-20.5167,57.5167,78.80,83,10.36,40,Souillac
3,Port Elizabeth,ZA,1613178780,-33.9180,25.5701,75.20,94,14.97,40,Port Elizabeth
4,Kapaa,US,1613178796,22.0752,-159.3190,78.01,57,8.05,1,Kapaʻa


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fit_weather_df.iterrows()]
locations = fit_weather_df[["lat", "lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))